In [1]:
pip install contractions transformers torch tqdm


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: /home/c14210092/Hallo_portrait_image_animation/halloenv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import torch
from transformers import BertTokenizer, BertModel
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers, activations, regularizers, initializers, constraints
import tensorflow.keras.backend as K
from contractions import fix
from nltk.tokenize import word_tokenize
import nltk

2025-06-07 12:06:52.104125: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-07 12:06:52.118545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749298012.136441 4184188 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749298012.141739 4184188 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749298012.155480 4184188 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

In [2]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/c14210092/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
class SquashActivation(layers.Layer):
    def call(self, inputs):
        s_squared_norm = tf.reduce_sum(
            tf.square(inputs), axis=-1, keepdims=True) + tf.keras.backend.epsilon()
        scale = tf.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
        return scale * inputs

    def get_config(self):
        return super().get_config()


class Capsule(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, share_weights=True,
                 initializer='glorot_uniform', activation=None, regularizer=None, constraint=None, **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        self.activation = activations.get(activation)
        self.regularizer = regularizers.get(regularizer)
        self.initializer = initializers.get(initializer)
        self.constraint = constraints.get(constraint)

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer=self.initializer, regularizer=self.regularizer,
                                     constraint=self.constraint, trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer=self.initializer, regularizer=self.regularizer,
                                     constraint=self.constraint, trainable=True)
        self.built = True

    def call(self, inputs):
        if self.share_weights:
            u_hat_vectors = K.conv1d(inputs, self.W)
        else:
            u_hat_vectors = K.local_conv1d(inputs, self.W, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        u_hat_vectors = K.reshape(
            u_hat_vectors, (batch_size, input_num_capsule, self.num_capsule, self.dim_capsule))
        u_hat_vectors = K.permute_dimensions(u_hat_vectors, (0, 2, 1, 3))
        routing_weights = K.zeros_like(u_hat_vectors[:, :, :, 0])

        for i in range(self.routings):
            capsule_weights = K.softmax(routing_weights, 1)
            outputs = K.batch_dot(capsule_weights, u_hat_vectors, [2, 2])
            if K.ndim(outputs) == 4:
                outputs = K.sum(outputs, axis=1)
            if i < self.routings - 1:
                outputs = K.l2_normalize(outputs, -1)
                routing_weights = K.batch_dot(outputs, u_hat_vectors, [2, 3])
                if K.ndim(routing_weights) == 4:
                    routing_weights = K.sum(routing_weights, axis=1)

        return self.activation(outputs)

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

    def get_config(self):
        config = super(Capsule, self).get_config()
        config.update({
            'num_capsule': self.num_capsule, 'dim_capsule': self.dim_capsule,
            'routings': self.routings, 'share_weights': self.share_weights,
            'activation': activations.serialize(self.activation),
            'initializer': initializers.serialize(self.initializer),
            'regularizer': regularizers.serialize(self.regularizer),
            'constraint': constraints.serialize(self.constraint)
        })
        return config

In [4]:
class SimpleToxicClassifier:
    def __init__(self):
        self.models = {}
        self.tokenizer = None
        self.bert_tokenizer = None
        self.bert_model = None
        self.bert_embeddings = None
        self.max_len = 200
        self.device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu')
        self.label_names = ['toxic', 'severe_toxic',
                            'obscene', 'threat', 'insult', 'identity_hate']
        print(f"Using device: {self.device}")

    def load_tokenizer(self, tokenizer_path):
        """Load the tokenizer for CNN models"""
        with open(tokenizer_path, 'rb') as f:
            self.tokenizer = pickle.load(f)
        print(f"Tokenizer loaded from {tokenizer_path}")

    def load_bert_embeddings(self, embeddings_path):
        """Load pre-computed BERT embeddings"""
        self.bert_embeddings = np.load(embeddings_path)
        print(f"BERT embeddings loaded: {self.bert_embeddings.shape}")

    def init_bert(self, model_name='bert-base-uncased'):
        """Initialize BERT embedding computation"""
        self.bert_tokenizer = BertTokenizer.from_pretrained(model_name)
        self.bert_model = BertModel.from_pretrained(model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.bert_model.eval()
        print(f"BERT model {model_name} initialized")

    def load_model(self, name, path, model_type):
        """Load a single model"""
        custom_objects = {'Capsule': Capsule,
                          'SquashActivation': SquashActivation}

        if 'capsule' in model_type.lower():
            model = load_model(path, custom_objects=custom_objects)
        else:
            model = load_model(path)

        self.models[name] = {'model': model, 'type': model_type}
        print(f"Model {name} loaded successfully")

    def preprocess_text(self, text):
        """Clean and preprocess text for CNN models"""
        if not isinstance(text, str):
            text = str(text)

        text = text.lower()
        text = re.sub(r'http\S+|www\S+', 'url', text)
        text = re.sub(r'@\w+', 'username', text)
        text = re.sub(r'[^a-z\s]', '', text)
        text = re.sub(r'\d+', 'number', text)
        text = fix(text)
        text = ' '.join(word_tokenize(text))
        return text

    def get_bert_embedding(self, text):
        """Get BERT embedding for text"""
        if self.bert_model is None:
            raise ValueError("BERT model not initialized")

        inputs = self.bert_tokenizer(text, return_tensors='pt', truncation=True,
                                     padding=True, max_length=self.max_len).to(self.device)

        with torch.no_grad():
            outputs = self.bert_model(**inputs)

        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    def classify_text(self, text):
        """Main classification function"""
        print(f"\nOriginal Text:\n{text}\n")
        print(self.label_names)

        # Preprocess for CNN models
        processed_text = self.preprocess_text(text)
        cnn_input = None
        if self.tokenizer:
            seq = self.tokenizer.texts_to_sequences([processed_text])
            cnn_input = pad_sequences(
                seq, maxlen=self.max_len, padding='post', truncating='post')

        # Get BERT embedding
        bert_input = None
        if self.bert_model:
            bert_input = self.get_bert_embedding(text)

        # Store all predictions - explicitly initialize as dictionary
        all_predictions = dict()

        # Make predictions with all models
        for name, model_info in self.models.items():
            model = model_info['model']
            model_type = model_info['type']

            # Initialize prediction variable as None
            prediction_result = None

            try:
                if 'bert' in model_type.lower():
                    if bert_input is not None:
                        prediction_result = model.predict(
                            bert_input, verbose=0)
                    else:
                        print(
                            f"BERT model {name} requires BERT initialization")
                        continue
                else:
                    if cnn_input is not None:
                        prediction_result = model.predict(cnn_input, verbose=0)
                    else:
                        print(f"CNN model {name} requires tokenizer")
                        continue

                # Format prediction
                if prediction_result.ndim > 1 and prediction_result.shape[0] == 1:
                    prediction_result = prediction_result[0]

                # Convert to binary predictions
                binary_pred = (prediction_result > 0.5).astype(int)

                # Store both raw probabilities and binary predictions
                all_predictions[name] = {
                    'probabilities': prediction_result.tolist(),
                    'binary': binary_pred.tolist(),
                    'labels': dict(zip(self.label_names, binary_pred.tolist()))
                }

                print(f"{name} Prediction: {binary_pred}")

            except Exception as e:
                print(f"Error with {name}: {str(e)}")
                all_predictions[name] = {'error': str(e)}

        return all_predictions

In [5]:
# Setup function
def setup_classifier(tokenizer_path=None, bert_embeddings_path=None, use_bert=True):
    """Setup function"""
    classifier = SimpleToxicClassifier()

    if tokenizer_path:
        classifier.load_tokenizer(tokenizer_path)

    if bert_embeddings_path:
        classifier.load_bert_embeddings(bert_embeddings_path)

    if use_bert and not bert_embeddings_path:
        classifier.init_bert()

    return classifier

In [6]:
# Setup classifier
classifier = setup_classifier(
    # Update with your path
    tokenizer_path='tokenizer.pkl',
    use_bert=True
)
# Load models
classifier.load_model(
    'CnnLSTM', 'model_epoch_02.h5', 'cnn_lstm')
classifier.load_model(
    'CnnLSTM_Capsule', 'model_dim16_routing3_epoch02.h5', 'cnn_lstm_capsule')
classifier.load_model(
    'BERT_BiGRU', 'bert_bgru_epoch_09.h5', 'bert_bigru')
classifier.load_model('BERT_BiGRU_Capsule',
                      'model_dim8_routing3_epoch29.h5', 'bert_bigru_capsule')

Using device: cuda
Tokenizer loaded from tokenizer.pkl
BERT model bert-base-uncased initialized


I0000 00:00:1749298016.515963 4184188 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15716 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:31:00.0, compute capability: 8.6
I0000 00:00:1749298016.519455 4184188 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 4909 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:4b:00.0, compute capability: 8.6


Model CnnLSTM loaded successfully
Model CnnLSTM_Capsule loaded successfully
Model BERT_BiGRU loaded successfully
Model BERT_BiGRU_Capsule loaded successfully


In [7]:
a = classifier.classify_text(
    "and you are a motherfucking asshole,suck your dick,you dirty son of a whore")


Original Text:
and you are a motherfucking asshole,suck your dick,you dirty son of a whore

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


I0000 00:00:1749298018.632734 4184522 cuda_dnn.cc:529] Loaded cuDNN version 90501


CnnLSTM Prediction: [1 0 1 0 1 0]
CnnLSTM_Capsule Prediction: [1 1 1 0 1 0]
BERT_BiGRU Prediction: [1 0 1 0 1 0]
BERT_BiGRU_Capsule Prediction: [1 1 1 0 1 0]


In [8]:
b = classifier.classify_text(
    "YOU NEED SUM PUSSY GO PAY A GIRL SHE WILL TAKE CARE OF YOU, I SAY THIS BECUASE MOST PPL HERE ARE DORKS AND VIRGINS THATS NEVER EVEN TOUCHED THE A FEMALE")


Original Text:
YOU NEED SUM PUSSY GO PAY A GIRL SHE WILL TAKE CARE OF YOU, I SAY THIS BECUASE MOST PPL HERE ARE DORKS AND VIRGINS THATS NEVER EVEN TOUCHED THE A FEMALE

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
CnnLSTM Prediction: [1 0 1 0 1 0]
CnnLSTM_Capsule Prediction: [1 0 0 0 0 0]
BERT_BiGRU Prediction: [1 0 1 0 1 0]
BERT_BiGRU_Capsule Prediction: [1 0 1 0 1 0]


In [9]:
b

{'CnnLSTM': {'probabilities': [0.9433300495147705,
   0.028500886633992195,
   0.6154075860977173,
   0.03338758274912834,
   0.534965455532074,
   0.06155205890536308],
  'binary': [1, 0, 1, 0, 1, 0],
  'labels': {'toxic': 1,
   'severe_toxic': 0,
   'obscene': 1,
   'threat': 0,
   'insult': 1,
   'identity_hate': 0}},
 'CnnLSTM_Capsule': {'probabilities': [0.8507922291755676,
   0.007761974819004536,
   0.3601892292499542,
   0.008233538828790188,
   0.30752354860305786,
   0.026722833514213562],
  'binary': [1, 0, 0, 0, 0, 0],
  'labels': {'toxic': 1,
   'severe_toxic': 0,
   'obscene': 0,
   'threat': 0,
   'insult': 0,
   'identity_hate': 0}},
 'BERT_BiGRU': {'probabilities': [0.9506295323371887,
   0.3133198916912079,
   0.8824242949485779,
   0.030847379937767982,
   0.8201826810836792,
   0.16107206046581268],
  'binary': [1, 0, 1, 0, 1, 0],
  'labels': {'toxic': 1,
   'severe_toxic': 0,
   'obscene': 1,
   'threat': 0,
   'insult': 1,
   'identity_hate': 0}},
 'BERT_BiGRU_Cap

In [10]:
def my_function(message):  # hanya untuk cek
    print("Hello from a function")
    testAdd = "ok"
    message = message + testAdd
    return message

In [11]:
# Initialize classifier globally (add this at the top of your script)
classifier = SimpleToxicClassifier()
# Load your models here
classifier.load_tokenizer('tokenizer.pkl')
classifier.load_model(
    name='CnnLSTM',
    path='model_epoch_02.h5',
    model_type='cnn_lstm'
)

# CNN LSTM Capsule Model
classifier.load_model(
    name='CnnLSTM_Capsule',
    path='model_dim16_routing3_epoch02.h5',
    model_type='cnn_lstm_capsule'
)

# BERT BiGRU Model
classifier.load_model(
    name='BERT_BiGRU',
    path='bert_bgru_epoch_09.h5',
    model_type='bert_bigru'
)

# BERT BiGRU Capsule Model
classifier.load_model(
    name='BERT_BiGRU_Capsule',
    path='model_dim8_routing3_epoch29.h5',
    model_type='bert_bigru_capsule'
)

classifier.init_bert()

Using device: cuda
Tokenizer loaded from tokenizer.pkl


Model CnnLSTM loaded successfully
Model CnnLSTM_Capsule loaded successfully
Model BERT_BiGRU loaded successfully
Model BERT_BiGRU_Capsule loaded successfully
BERT model bert-base-uncased initialized


In [12]:
def score_comment(comment, username, id, user_mention):
    """Updated to use global classifier - maintains original function signature"""
    global classifier

    # Get predictions from the classifier
    results = classifier.classify_text(comment)

    check = False
    header = (
        f"**Message:** {comment}\n"
        f"**Username:** {username}\n"
        f"**ID:** {id}\n"
        f"{user_mention}\n\n"
        "```"
    )

    labels = ['toxic', 'severe_toxic', 'obscene',
              'threat', 'insult', 'identity_hate']
    body = ''

    # Process results from each model
    for model_name, prediction_data in results.items():
        # Skip models that had errors
        if 'error' in prediction_data:
            body += f"Model: {model_name}\n"
            body += f"Error: {prediction_data['error']}\n\n"
            continue

        body += f"Model: {model_name}\n"
        body += f"{'Label':<16}\t{'Flag':<8}\tScore\n"
        body += f"{'-'*40}\n"

        probabilities = prediction_data['probabilities']
        binary_predictions = prediction_data['binary']

        for idx, label in enumerate(labels):
            if idx < len(probabilities):  # Safety check
                score = probabilities[idx]
                is_flagged = binary_predictions[idx] == 1
                body += f"{label:<16}\t{str(is_flagged):<8}\t{score:.2f}\n"
                if is_flagged:
                    check = True
        body += '\n'

    if not check:
        return '**FALSE**'  # No toxic content detected
    else:
        return header + body + '```'

In [13]:
from datetime import datetime, timedelta
from discord.utils import utcnow

In [ ]:
import asyncio
import nest_asyncio
import discord

# setting beberapa parameter discord bot nya agar bisa mengirim/ membaca message
intents = discord.Intents.default()
intents.messages = True
intents.message_content = True

client = discord.Client(intents=intents)

# inisialisasi token bot
BOT_TOKEN = 'xxx'


async def send_hello_world(channel, message):
    if message == '':  # hanya untuk message yang memang tidak ada message :D
        return

    try:
        await channel.send(message)  # melihat apakah message sudah di send
    except Exception as e:
        # mengembalikan error jika terjadi error
        print(f"Error sending message: {e}")


@client.event
async def on_ready():
    print(f'Logged in as {client.user.name}')


@client.event
async def on_message(message):
    if message.author == client.user:
        return

    # if message.content.startswith('$hello'):
    #     channel = message.channel
    #     task = client.loop.create_task(send_hello_world(channel, 'hellooooooooooooo' ))

    if client.user in message.mentions and 'general' in str(message.channel):
        channel = message.channel

        # cuma buat mengecek apa bot aktif, kalau iya bot akan mengirim reply jika di tag
        reply = my_function('haiiiiiiiiiiiasdj,kasdjl')

        task = client.loop.create_task(send_hello_world(channel, reply))
        await task

    if 'general' in str(message.channel):
        content = message.content
        user_name = message.author.name
        user_id = message.author.id
        user_mention = message.author.mention

        try:
            reply = score_comment(
                content, user_name, user_id, user_mention)

            if reply != '**FALSE**':  # toxic content found

                # ⏳ Timeout the user
                timeout_until = utcnow() + timedelta(minutes=.1)
                await message.author.edit(timed_out_until=timeout_until, reason="Toxic message detected")
                print(f"User {user_name} timed out until {timeout_until}")

                # 🚫 Delete the message
                await message.delete()

                # 🛡️ Optional: Ban the user
                # await message.author.ban(reason="Toxic message", delete_message_days=1)

                # 📢 Report to 'reports' channel
                reply_channel = client.get_channel(1251881766256509049)
                await reply_channel.send(reply)

        except Exception as e:
            print(f"Error processing message: {e}")


async def start_bot():  # untuk mengaktifkan bot dengan token yang sudah di inisialisasi di atas
    await client.start(BOT_TOKEN)

# Run the bot using asyncio in a Jupyter Notebook
nest_asyncio.apply()


async def run_bot():
    await start_bot()

loop = asyncio.get_event_loop()
# bagian sini itu intinya bot nya bisa nyala selama servernya nyala sih, mau di Jupyter/collab mirip2. Kalau di jupyter, server dari laptop sendiri. Kalau collab, ya dari google nya
loop.run_until_complete(run_bot())
# Aku sudah nyoba kalau ini tidak dilakukan, bot kadang mati sendiri meski server masih nyala. Selama code ini kupakai, tidak pernah mati unless memang di stop notebooknya.

# To stop the bot later (optional)
client.close()
loop.run_until_complete(client.logout())

Logged in as Toxic Detection Bot

Original Text:
test

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


CnnLSTM Prediction: [0 0 0 0 0 0]


CnnLSTM_Capsule Prediction: [0 0 0 0 0 0]
BERT_BiGRU Prediction: [0 0 0 0 0 0]
BERT_BiGRU_Capsule Prediction: [0 0 0 0 0 0]

Original Text:
fuckk you

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
CnnLSTM Prediction: [1 0 0 0 0 0]
CnnLSTM_Capsule Prediction: [0 0 0 0 0 0]
BERT_BiGRU Prediction: [1 0 1 0 1 0]
BERT_BiGRU_Capsule Prediction: [1 0 1 0 1 0]
User mixlin timed out until 2025-06-07 12:08:32.433773+00:00

Original Text:
fuck off

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
CnnLSTM Prediction: [1 0 1 0 1 0]
CnnLSTM_Capsule Prediction: [1 0 1 0 1 0]
BERT_BiGRU Prediction: [1 0 1 0 1 0]
BERT_BiGRU_Capsule Prediction: [1 0 0 0 0 0]
User mixlin timed out until 2025-06-07 12:08:48.780144+00:00

Original Text:
nigga

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
CnnLSTM Prediction: [1 0 0 0 0 0]
CnnLSTM_Capsule Prediction: [1 0 0 0 0 0]
BERT_BiGRU Prediction: [1 0 0 0 0 0]
BERT_BiGRU_Capsule Predicti